In [1]:
import torch
import pandas as pd
import numpy as np
import random
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from matplotlib import pyplot as plt


In [2]:
class cnn(nn.Module):
    def __init__(self):
        super().__init__()
        self.act = torch.nn.Tanh()
        self.linear1 = nn.Conv2d(4 , 1, kernel_size = (2,1), stride = 1, padding = 0)
        # self.linear2 = torch.nn.Linear(6,6)
        # self.linear3 = torch.nn.Linear(6,6) 
        # self.linear4 = torch.nn.Linear(6,1)
    def forward(self, X):
        X = self.linear1(X)
        # X = self.act(X)
        # X = self.linear2(X)
        # X = self.act(X)
        # X = self.linear3(X)
        # X = self.act(X)
        # X = self.linear4(X)
        return X

In [3]:
model = cnn()
print(model)

cnn(
  (act): Tanh()
  (linear1): Conv2d(4, 1, kernel_size=(2, 1), stride=(1, 1))
)


In [4]:
X = []
Y = []
for i in range(100000):
    x = np.random.randint(0,400,(4,1))
    x = np.tile(x,(1,4))
    X.append(x)
    y = [x[i]-x[0] for i in range(len(x))]
    Y.append(y)

# print(X.iloc[0])
# print(Y.iloc[0])

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define the CNN model
class SimpleCNN(nn.Module):
    def __init__(self, output_size):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1)  # Output: 8x4x4
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)  # Output: 16x4x4
        self.fc = nn.Linear(16 * 4 * 4, output_size)  # Fully connected layer to desired output size

    def forward(self, x):
        x = F.relu(self.conv1(x))  # Apply first convolution and activation
        x = F.relu(self.conv2(x))  # Apply second convolution and activation
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)  # Fully connected layer
        return x

# Example: Instantiate the model for a 1x4 output
output_size = 4  # Change to 3 for 1x3 output
model = SimpleCNN(output_size)

# Example input: A single 4x4 image with 1 channel
input_tensor = torch.tensor([[[[1, 2, 3, 4],
                                [1, 2, 3, 4],
                                [1, 2, 3, 4],
                                [1, 2, 3, 4]]]], dtype=torch.float32)  # Shape: [1, 1, 4, 4]

# Forward pass
output = model(input_tensor)
print(f"Input shape: {input_tensor.shape}")
print(f"Output shape: {output.shape}")
print(f"Output: {output}")


Input shape: torch.Size([1, 1, 4, 4])
Output shape: torch.Size([1, 4])
Output: tensor([[ 0.1109,  0.0362,  0.0928, -0.0124]], grad_fn=<AddmmBackward0>)


In [6]:
class MyDataset(Dataset):
    def __init__(self, X, Y):
        super().__init__()
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)
        self.Y = (torch.tensor(Y, dtype = torch.float32)).unsqueeze(2)
        # self.Y = (torch.tensor(Y, dtype = torch.float32).long()).unsqueeze(2).unsqueeze(1)
        # self.Y = (torch.tensor(Y, dtype = torch.float32).long())
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [7]:
Data = MyDataset( X,Y)
# print(Data.X)
dataset = DataLoader(Data, batch_size= 10000, shuffle=True)

loss = nn.MSELoss(reduction='sum')
opt = torch.optim.Adam(model.parameters(), lr=0.001)


C:\Users\arpit\AppData\Local\Temp\ipykernel_16872\901273907.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:204.)
  self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)


In [8]:
def train_loop(dataloader, model, loss_fn, optimizer, batch_size, ls):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    # ls = []
    for batch, (X, Y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        # print(pred)
        # print(Y)
        loss = loss_fn(pred, Y)
        # break
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        ls.append(loss.item())
        plt.plot(ls)
        plt.savefig("dam.png")
        

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        return ls

In [9]:
# import torch
# import torch.nn as nn

# # Example data
# batch_size, x, units, y = 1000, 1, 3, 1
# pred = torch.randn(batch_size, x, units, y, requires_grad=True)  # Example predictions
# Y = torch.randn(batch_size, x, units, y)  # Example targets

# # Define loss function and optimizer
# loss_fn = nn.MSELoss()
# optimizer = torch.optim.Adam([pred], lr=0.01)

# # Compute loss for the entire batch

# loss += loss_fn(pred, Y)
# print(f"Loss: {loss.item()}")

# # Backward pass and optimization
# loss.backward()
# optimizer.step()
# optimizer.zero_grad()


In [10]:
# for batch, (X, Y) in enumerate(dataset):
#         # Compute prediction and loss
#         print(X.shape)
#         pred = model(X)
#         batch,x,units,y = pred.shape
#         print(batch, units)
#         for i in range(batch):
#                 for j in range(units):
#                         print(pred[i][0][j][0])
#         break
#         # print(pred)
#         # print(Y)

In [ ]:
ls = []
for t in range(500):
    print(f"Epoch: {t+1}\n------------------")
    ls  = train_loop(dataset,model,loss,opt,1000,ls)
    # with torch.no_grad():
        # y = pd.DataFrame(Parabola_model(X_tensor),columns=['y'])
    # print(type(y))
        # df = pd.concat([X,y],axis=1)
        # df.plot(kind='scatter',x='X',y='y',s = 1.5)
        # plt.savefig('C:/Projects/AI-learning/Py-torch proto/training.jpg')

Epoch: 1
------------------


c:\Users\arpit\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:530: UserWarning: Using a target size (torch.Size([10000, 4, 1, 4])) that is different to the input size (torch.Size([10000, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
model(torch.tensor([[4,5,7,66],[4,4,4,4]], dtype = torch.float32).unsqueeze(2))

tensor([[[ 22.8439],
         [ 26.9780],
         [121.3525]]], grad_fn=<SqueezeBackward1>)